# Connectome Overview, Basic Statistics, and Visualizations #

This notebook provides an overview of the 14 connectomes available in the connectocross repository, along with visualizations of each one and some basic statistics

In [ ]:
import os
import sys
sys.path.append("../")
import graspologic as gr
import networkx as nx
from graph import GraphIO
import pandas as pd

## Ciona Intestinalis Larval Connectome ##

This is a very detailed connectome of the ascidian tadpole larva create via careful EM imaging. Multiple hermaphroditic individuals were imaged. The original study was looking for topological asymmetries between hemispheres in Ciona's central nervous system (CNS). The connectome includes mostly CNS Neurons.

{cite}`ryan_cns_2016`

In [ ]:
path = '../json_connectomes/ciona.json'
connectome, _, _, _ = GraphIO.load(path)

Now we generate a 2-dimmensional node layout using Graspologic's `layout_tnse`. This function creates a layout that shows
the community structure of the data.

In [ ]:
undirected_connectome = nx.convert_node_labels_to_integers(connectome.to_undirected())
undirected_connectome, layout = gr.layouts.layout_tsne(undirected_connectome,
                                                       perplexity=20,
                                                       n_iter=1000000)

Next we must assign a color to the nodes in the graph. Here we base the colors on node "Side" attributes. Left is Blue,
Right is Red, and Medial is Purple. Nodes without a side attribute are grey.

In [ ]:

color_map = {}
side_map = {'L': '#FF0000', 'DL': '#B22222', 'VL': '#F08080',
            'R': '#0000FF', 'DR': '#00008B', 'VR': '#87CEEB',
            'DM': '#800080', 'D': '#800080', 'VM': '#BA55D3',
            None: '#A9A9A9'}

for node in undirected_connectome.nodes(data=True):
    try:
        side = node[1]['Side'].strip()
    except KeyError:
        side = None
    color_map[node[0]] = side_map[side]

Now we will show the generated and colored layout

In [ ]:
gr.layouts.show_graph(undirected_connectome, layout, color_map,
                      vertex_shape='.', vertex_alpha=1.,
                      edge_alpha=.2, edge_line_width=2.)

Now lets display some basic graph statistics:

In [ ]:
data = {'Number of Nodes': [],
        'Number of Edges': [],
        'Density': [],
        'Max Out Degree': [],
        'Max In Degree': []}
num_nodes = len(connectome.nodes)
data['Number of Nodes'].append(num_nodes)
num_edges = len(connectome.edges)
data['Number of Edges'].append(num_edges)
data['Density'].append(num_edges / (num_nodes * (num_nodes - 1)))
data['Max Out Degree'].append(max(connectome.out_degree, key=lambda x: x[1])[1])
data['Max In Degree'].append(max(connectome.in_degree, key=lambda x: x[1])[1])
df = pd.DataFrame.from_dict(data)
df.index = ['Ciona Intestinalis',]
df.head()

## C. Elegans: Connectomes Over Development ##

This dataset seeks to elucidate how connectomes change over development. Eight different EM connectomes were constructed
from C. Elegans Hermaphrodites at evenly spaced time intervals from birth until adulthood.

{cite}`witvliet_connectomes_2020`

When loading the connectomes below, we must flatten the multigraphs. This is because chemical and electrical synapses are included, so some nodes
may have two pairs of edges between them. Here we rectify this by summing weights of multiedges (the default mode for `GraphIO.flatten_multigraph`)

In [ ]:
# load some elegans witvilet data
base = '../json_connectomes/witvilet/'
graph_files = sorted(os.listdir(base))
connectomes = []
for f in graph_files:
    connectomes.append(GraphIO.flatten_multigraph(GraphIO.load(os.path.join(base, f))[0]))

We follow the same steps as for Ciona (above) to generate visualizations, only here we loop over all 8 connectomes.

In [ ]:
layouts = [None]*len(connectomes)
undirected_cleaned = [None]*len(connectomes)
for i, connectome in enumerate(connectomes):
    temp_g = nx.convert_node_labels_to_integers(connectome.to_undirected())
    undirected_cleaned[i], layouts[i] = gr.layouts.layout_tsne(temp_g,
                                                               perplexity=20,
                                                               n_iter=10000000)


In [ ]:
color_maps = [{} for i in range(len(connectome))]
hemi_map = {'left': '#FF0000',
            'right': '#0000FF',
            None: '#A9A9A9'}
for i, connectome in enumerate(undirected_cleaned):
    for node in connectome.nodes(data=True):
        try:
            side = node[1]['hemisphere'].strip()
        except (AttributeError, KeyError):
            side = None
        color_maps[i][node[0]] = hemi_map[side]

In [ ]:
for i in range(len(layouts)):
    print('C. Elegans, Stage ' + str(i + 1))
    gr.layouts.show_graph(undirected_cleaned[i].to_undirected(), layouts[i], color_maps[i],
                          vertex_shape='.', vertex_alpha=1.,
                          edge_alpha=.2, edge_line_width=2.,)

Once again, lets get some basic stats for each connectome.

In [ ]:
data = {'Number of Nodes': [],
        'Number of Edges': [],
        'Density': [],
        'Max Out Degree': [],
        'Max In Degree': []}
for i, connectome in enumerate(connectomes):
    num_nodes = len(connectome.nodes)
    data['Number of Nodes'].append(num_nodes)
    num_edges = len(connectome.edges)
    data['Number of Edges'].append(num_edges)
    data['Density'].append(num_edges / (num_nodes * (num_nodes - 1)))
    data['Max Out Degree'].append(max(connectome.out_degree, key=lambda x: x[1])[1])
    data['Max In Degree'].append(max(connectome.in_degree, key=lambda x: x[1])[1])

df = pd.DataFrame.from_dict(data)
df.index = ['C. Elegans: stage 1',
            'C. Elegans: stage 2',
            'C. Elegans: stage 3',
            'C. Elegans: stage 4',
            'C. Elegans: stage 5',
            'C. Elegans: stage 6',
            'C. Elegans: stage 7',
            'C. Elegans: stage 8',]
df.head(8)

## C. Elegens: Worm Wiring Connectome, Hermaphrodite and Male ##

This dataset contains full, detailed, connectomes for both C. Elegans sexes. Data for chemical and electrical synapses is contained in this repository,
but here we look more deeply at the chemical connectome.

{cite}`cook_whole-animal_2019`

Again, visualization and analysis is done as before.

In [ ]:
# load chemical synapse connectome files
path_h = '../json_connectomes/worm_wiring/connectome/Hermaphrodite/0.json'
path_m = '../json_connectomes/worm_wiring/connectome/Male/0.json'
connectomes = []
connectomes.append(GraphIO.load(path_h)[0])
connectomes.append(GraphIO.load(path_m)[0])


In [ ]:
layouts = [None]*len(connectomes)
undirected_cleaned = [None]*len(connectomes)
for i, connectome in enumerate(connectomes):
    temp_g = nx.convert_node_labels_to_integers(connectome.to_undirected())
    undirected_cleaned[i], layouts[i] = gr.layouts.layout_tsne(temp_g,
                                                               perplexity=20,
                                                               n_iter=10000000)

In [ ]:
color_maps = [{} for i in range(len(connectome))]
hemi_map = {'left': '#FF0000',
            'right': '#0000FF',
            None: '#A9A9A9'}
for i, connectome in enumerate(undirected_cleaned):
    for node in connectome.nodes(data=True):
        try:
            side = node[1]['hemisphere'].strip()
        except (AttributeError, KeyError):
            side = None
        color_maps[i][node[0]] = hemi_map[side]

In [ ]:
labels = ['Hermaphrodite Connectome', 'Male Connectome']
for i in range(len(layouts)):
    print(labels[i])
    gr.layouts.show_graph(undirected_cleaned[i].to_undirected(),
                          layouts[i], color_maps[i],
                          vertex_shape='.', vertex_alpha=1.,
                          edge_alpha=.2, edge_line_width=2.)


Now let's get the basic graph statistics.

In [ ]:
data = {'Number of Nodes': [],
        'Number of Edges': [],
        'Density': [],
        'Max Out Degree': [],
        'Max In Degree': []}
for i, connectome in enumerate(connectomes):
    num_nodes = len(connectome.nodes)
    data['Number of Nodes'].append(num_nodes)
    num_edges = len(connectome.edges)
    data['Number of Edges'].append(num_edges)
    data['Density'].append(num_edges / (num_nodes * (num_nodes - 1)))
    data['Max Out Degree'].append(max(connectome.out_degree, key=lambda x: x[1])[1])
    data['Max In Degree'].append(max(connectome.in_degree, key=lambda x: x[1])[1])

df = pd.DataFrame.from_dict(data)
df.index = ['C. Elegans: Worm Wiring, Hermaphrodite, Chemical',
            'C. Elegans: Worm Wiring, Male, Chemical']
df.head(2)

## Drosophila (FAFB Dataset): Mushroom Body ##

Using new High Throughput Electron Micrography data, this dataset contains a subset of the whole Fly Brain Connectome, namely the mushroom body. This dataset is much smaller than the others, because it focuses
on a specific neural circuit.

{cite}`zheng_complete_2018`


In [ ]:
path = '../json_connectomes/fafb.json'
connectome, _, _, _ = GraphIO.load(path)

In [ ]:
undirected_cleaned, layout = gr.layouts.layout_tsne(connectome.to_undirected(),
                                                    perplexity=20, n_iter=1000)

In [ ]:
color_map = {}
side_map = {'L': '#FF0000',
            'R': '#0000FF',
            None: '#A9A9A9'}

for node in connectome.nodes(data=True):
    try:
        side = node[1]['Side'].strip()
    except (KeyError, AttributeError):
        side = None
    color_map[node[0]] = side_map[side]

In [ ]:
gr.layouts.show_graph(undirected_cleaned, layout, color_map,
                      vertex_shape='.', vertex_alpha=1.,
                      edge_alpha=.2, edge_line_width=2.)

FAFB Basic Statistics:

In [ ]:
num_nodes = len(connectome.nodes)
num_edges = len(connectome.edges)
density = num_edges / (num_nodes * (num_nodes - 1))
data = {'Number of Nodes': [num_nodes],
        'Number of Edges': [num_edges],
        'Density': [density],
        'Max Out Degree': max(connectome.out_degree, key=lambda x: x[1])[1],
        'Max In Degree': max(connectome.in_degree, key=lambda x: x[1])[1]}
df = pd.DataFrame.from_dict(data)
df.index = ['FAFB Connectome']
df.head()

## Drosophila Hemibrain Connectome ##

Largest synaptic level connectome ever recorded, includes large portion of a fly's CNS. Imaged using a combination of EM and
light microscopy.

{cite}`xu_connectome_2020`

This connectome is much larger than the others, so some steps will take longer to run.

In [ ]:
path = '../json_connectomes/neuPrint_graphs.json'
connectome, _, _, _ = GraphIO.load(path)

We can't visualize this connectome using grapological layouts becuase it is too large! We can still get the basic stats though:


In [ ]:
num_nodes = len(connectome.nodes)
num_edges = len(connectome.edges)
density = num_edges / (num_nodes * (num_nodes - 1))
data = {'Number of Nodes': [num_nodes],
        'Number of Edges': [num_edges],
        'Density': [density],
        'Max Out Degree': max(connectome.out_degree, key=lambda x: x[1])[1],
        'Max In Degree': max(connectome.in_degree, key=lambda x: x[1])[1]}
df = pd.DataFrame.from_dict(data)
df.index = ['Drosophila Hemibrain Connectome']
df.head()
